In [107]:
 #import libraries 
import pandas as pd     
import numpy as np 
import spacy
nlp=spacy.load('en_core_web_sm')
import re
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer 
from nltk.stem import WordNetLemmatizer  
from sklearn.feature_extraction.text import TfidfVectorizer 


In [108]:
#Import data
train = pd.read_csv('C:/Users/School EC/Downloads/classification-predict-streamlit-template-0cac363f0638641ddb35a3d30c4756bf4d25e7ef/Train.csv') 
test = pd.read_csv('C:/Users/School EC/Downloads/classification-predict-streamlit-template-0cac363f0638641ddb35a3d30c4756bf4d25e7ef/Test.csv')

In [109]:
test.head()

,message,tweetid
0,Europe will now be looking to China to make su...,169760
1,Combine this with the polling of staffers re c...,35326
2,"The scary, unimpeachable evidence that climate...",224985
3,@Karoli @morgfair @OsborneInk @dailykos \nPuti...,476263
4,RT @FakeWillMoore: 'Female orgasms cause globa...,872928


In [110]:
train.head()

,sentiment,message,tweetid
0,1,PolySciMajor EPA chief doesn't think carbon di...,625221
1,1,It's not like we lack evidence of anthropogeni...,126103
2,2,RT @RawStory: Researchers say we have three ye...,698562
3,1,#TodayinMaker# WIRED : 2016 was a pivotal year...,573736
4,1,"RT @SoyNovioDeTodas: It's 2016, and a racist, ...",466954


In [111]:
train.shape


(15819, 3)

In [112]:
test.shape

(10546, 2)

In [113]:
def preprocess_tweet_text(tweet): 
  
 tweet = str(tweet).lower()    
# Remove urls
 tweet = re.sub(r"http\S+|www\S+|https\S+", '', tweet, flags=re.MULTILINE)
# Remove user @ references and '#' from tweet
 tweet = re.sub(r'\@\w+|\#','', tweet)
# Remove punctuations
 tweet = tweet.translate(str.maketrans('', '', string.punctuation))
# Remove stopwords
 tweet_tokens = word_tokenize(tweet)
 filtered_words = [word for word in tweet_tokens if word not in stopwords.words('english')]

 porter = PorterStemmer()
 stemmed_words = [porter.stem(w) for w in filtered_words]
 lemmatizer = WordNetLemmatizer()
 lemma_words = [lemmatizer.lemmatize(word, pos='a') for word in stemmed_words]

 return " ".join(filtered_words) 

In [114]:
preprocess_tweet_text(train['message'])

'0 polyscimajor epa chief doesnt think carbon di 1 like lack evidence anthropogeni 2 rt researchers say three ye 3 todayinmaker wired 2016 pivotal year 4 rt 2016 racist 15814 rt took material 15815 rt climate change could b 15816 notiven rt nytimesworld trump act 15817 rt hey liberals climate chan 15818 rt climate c name message length 15819 dtype object'

In [174]:
#def int_to_string(sentiment):
    #if sentiment == -1:
        #return "Anti"
    #elif sentiment == 0:
        #return "Neutral"
    #elif sentiment == 1:
        #return "Pro"
    #else:
        #return "News"#

In [115]:
# -----------Now it's time to split the data and train Machine Learning models----------
# first import ML algorithms we'll use
# This predict is a classification problem so we'll  Classification models


from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import SGDClassifier 
from sklearn.metrics import confusion_matrix,classification_report
from sklearn.metrics import accuracy_score, f1_score, precision_recall_fscore_support 
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB 

from sklearn import metrics


In [116]:
y=train['sentiment'] 

X= train['message']

In [117]:
# Split the data using 80/20 split and don't set random state = 100, to select records randomly
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.1, random_state =10)

In [118]:
# Linear SVC:
text = Pipeline([('tfidf',TfidfVectorizer()),('classify',LinearSVC())])

In [119]:
text.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('tfidf',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('classify',
                 LinearSVC(C=1.0, class_weight=None, dual=True,
                           fit_intercept=True, intercept_scaling=1,
          

In [120]:
pred = text.predict(X_test)

In [121]:
X_unseen=f['message']

In [122]:
# Form a prediction set
y_prediction = text.predict(X_unseen)

In [123]:
# Print a classification report
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

          -1       0.73      0.53      0.61       132
           0       0.68      0.44      0.53       230
           1       0.79      0.88      0.83       840
           2       0.79      0.83      0.81       380

    accuracy                           0.77      1582
   macro avg       0.75      0.67      0.70      1582
weighted avg       0.77      0.77      0.76      1582



In [124]:
# Print the overall accuracy
print(metrics.accuracy_score(y_test,pred))

0.7743362831858407


In [131]:
# Now predict on the test data - which is the test file
# first extract the message column from it

newdata=test['message']

In [132]:
pred=text.predict(newdata)

In [133]:
 #set a series of dataframe for the tweet id column be extracting itfrom the test set

tweetid = test.set_index('tweetid')

In [134]:
tweet_index = tweetid.index

In [135]:
Submission_Table = {'tweetid': tweet_index, 'sentiment':np.round(pred, 0)}
submission = pd.DataFrame(Submission_Table)
submission = submission[['tweetid', 'sentiment']]

In [136]:
submission.set_index('tweetid')

,sentiment
tweetid,
169760,1
35326,1
224985,1
476263,1
872928,0
...,...
895714,1
875167,1
78329,2


In [137]:
submission.to_csv("classytweetsSiya.csv",index  = False) 